In [2]:
import h5py
from os import listdir
from os.path import isfile, join, isdir
import numpy as np

In [5]:
mypath = "Data 2"
onlydirs = [f for f in listdir(mypath) if isdir(join(mypath, f))]
h5filelist = []
samplelist = {}

In [6]:
onlydirs

['AIS M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-11',
 'AIS M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-16',
 'AIS M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-21',
 'AIS M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-26',
 'AIS M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-6']

In [7]:
for dirpath in onlydirs:
    aispath = join(mypath,dirpath)
    files = [f for f in listdir(aispath) if isfile(join(aispath, f))]
    h5files = [join(aispath,f) for f in files if f.endswith(".h5")]
    h5filelist += h5files

In [8]:
h5filelist

['Data 2\\AIS M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-11\\M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-11.h5',
 'Data 2\\AIS M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-16\\M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-16.h5',
 'Data 2\\AIS M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-21\\M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-21.h5',
 'Data 2\\AIS M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-26\\M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-26.h5',
 'Data 2\\AIS M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-6\\M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-6.h5']

In [9]:
for h5file in h5filelist:
    if h5py.is_hdf5(h5file):
        lala = h5py.File(h5file,"r")
        
        try:
            rois = [key for key in lala["Data"].keys()]
            lengths = []
            for roi in rois:
                length = lala["Data/"+roi+"/Physical"].attrs["AISPhysicalLength"]
                #length = lala["Data/"+roi+"/Pixeldata"].attrs["AISend"] - lala["Data/"+roi+"/Pixeldata"].attrs["AISstart"]
                lengths.append(length)
        
            lengths = np.array(lengths)
            seriesname = lala.attrs["seriesname"].split("_")[0]
            samplelist[seriesname] = np.append(samplelist.get(seriesname,np.array([])),lengths)
            print(lala.attrs["seriesname"] + "    " +str(lengths.mean()) + "   " + str(np.std(lengths)))
        except KeyError as e :
            print("File {0} doesnt hava data Object: {1}".format(h5file,e))
            pass
        
    else:
        print("File " + h5file + " doesn't conform with h5 standards")
        pass

M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-11    0.0195209474409   0.0062739260402
M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-16    0.027498780682   0.00892951149499
M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-21    0.0224701181265   0.00692341838094
M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-26    0.0248857920198   0.00630661631806
M1 development_p15_layer II-III_rb-betaIVSpectrin-488_gpNeuN-568_DAPI-405_630x_2-3-2-6    0.016435395353   0.00421304637229


In [10]:
for key,value in samplelist.items():
    print("Sample: {0}  Mean: {1} Std: {2} AIS: {3}".format(key,value.mean(),np.std(value),len(value)))

Sample: M1 development  Mean: 0.022370536933573614 Std: 0.007744414369454423 AIS: 150


In [22]:
from scipy.stats import ttest_ind, ttest_rel

t, p = ttest_ind(samplelist["TM349"],samplelist["TM340"])

In [24]:
pvalue = "{0:.10f}".format(p)
tvalue = "{0:.10f}".format(t)

In [14]:
samplelist["TM340"].size

40

In [25]:
print("T-Wert: {0} P-Wert: {1}".format(tvalue,pvalue))

T-Wert: 4.8303691249 P-Wert: 0.0000021309
